In [ ]:
import tensorflow as tf
import pickle
import numpy as np
import pandas as pd

In [ ]:
# # !pip install -U PyDrive
# from tensorflow.python.client import device_lib
# print(device_lib.list_local_devices())


In [ ]:
# from pydrive.auth import GoogleAuth
# from pydrive.drive import GoogleDrive

# gauth = GoogleAuth()
# # Try to load saved client credentials
# gauth.LoadCredentialsFile("mycreds.txt")
# if gauth.credentials is None:
#     # Authenticate if they're not there
#     gauth.LocalWebserverAuth()
# elif gauth.access_token_expired:
#     # Refresh them if expired
#     gauth.Refresh()
# else:
#     # Initialize the saved creds
#     gauth.Authorize()
# # Save the current credentials to a file
# gauth.SaveCredentialsFile("mycreds.txt")

# drive = GoogleDrive(gauth)


In [ ]:
#Import authentication libraries

# from google.colab import auth
# from pydrive.auth import GoogleAuth
# from pydrive.drive import GoogleDrive
# from oauth2client.client import GoogleCredentials

In [ ]:
# auth.authenticate_user()
# gauth = GoogleAuth()
# gauth.credentials = GoogleCredentials.get_application_default()
# drive = GoogleDrive(gauth)

In [ ]:
# folder_id='1uXkcRXuQ5fFG0S8mLW67ewj_jsqAb_Ff'

In [ ]:
# # Auto-iterate through all files in the midi folder.
# file_list = drive.ListFile({'q': "'{0}' in parents and trashed=false".format(folder_id)}).GetList()
# for file1 in file_list:
#   print('title: {0}, id: {1}'.format(file1['title'], file1['id']))

In [ ]:
# # Download all midi files
# for file1 in file_list:
#   file_id=file1['id']
#   file_title = file1['title']
#   drive.CreateFile({'id': file_id}).GetContentFile(file_title)
  

In [ ]:
from os import listdir
from os.path import isfile, join

In [ ]:
echo %cd%

In [ ]:
folder ='C:/Users/Gustavo/Documents/GitHub/GAN_project_PUCP/Midinet_VG/Model_type1'


In [ ]:
songs=[f for f in listdir(folder)]
songs=songs[15:-3]
songs

In [ ]:
data_songs=np.empty((0,16,128), int)
for song in songs:
    try:  
      song_opened = open(song,"rb")
      song_loaded= pickle.load(song_opened)
      #for track 1
      for i in range(len(song_loaded[0])):
        bar=np.empty((0,128), int)[None]
        for j in song_loaded[0][i]:
          note=np.zeros(128)[None][None]
          if type(j) == type(1.5):
            note[0,0,int(j)] = 1
          bar=np.append(bar,note,axis=1)
        data_songs =np.append(data_songs,bar, axis=0)
      print(np.array(data_songs).shape,'song=',song,np.array(song_loaded).shape)
    except:
        print('error parsing')
data_songs_prev =np.roll(data_songs,1,axis=0)
data_songs=data_songs[None]
data_songs_prev=data_songs_prev[None]

In [ ]:
print(data_songs.shape)
print(data_songs_prev.shape)
data_songs=np.transpose(data_songs, (1,0,2,3))#Real data, input for model
data_songs_prev=np.transpose(data_songs_prev, (1,0,2,3))#Real data, 1 previous bar, input for model
print(data_songs.shape)
print(data_songs_prev.shape)

In [ ]:
np.save('data_songs.npy', data_songs, allow_pickle=True, fix_imports=True)

In [ ]:
np.save('data_songs_prev.npy', data_songs_prev, allow_pickle=True, fix_imports=True)

In [1]:
import os
import scipy.misc
import numpy as np
from model import MidiNet #Llamando el modelo Utilizar model2 para eliminar silencios
from utils import pp, to_json, generation_test

# os.environ["CUDA_VISIBLE_DEVICES"] = '1'

import tensorflow as tf

flags = tf.app.flags
flags.DEFINE_integer("epoch", 50, "Epoch to train [20]")
flags.DEFINE_float("learning_rate", 0.00005, "Learning rate of for adam [0.0002]")
flags.DEFINE_float("beta1", 0.5, "Momentum term of adam [0.5]")
flags.DEFINE_integer("batch_size", 120, "The size of batch [120]")
flags.DEFINE_integer("output_w", 16, "The size of the output segs to produce [16]")
flags.DEFINE_integer("output_h", 128, "The size of the output note to produce [128]")
flags.DEFINE_integer("c_dim", 1, "Number of Midi track. [1]")
flags.DEFINE_string("checkpoint_dir", "checkpoint", "Directory name to save the checkpoints [checkpoint]")
flags.DEFINE_string("sample_dir", "samples", "Dircacectory name to save the image samples [samples]")
flags.DEFINE_string("dataset", "MidiNet_vg", "The name of dataset ")
flags.DEFINE_boolean("is_train", True, "True for training, False for testing [False]")
flags.DEFINE_boolean("is_crop", False, "True for training, False for testing [False]")
flags.DEFINE_boolean("generation_test", False, "True for generation_test, False for nothing [False]")
flags.DEFINE_string("gen_dir", "gen", "Directory name to save the generate samples [samples]")
FLAGS = flags.FLAGS

def main(_):
    pp.pprint(flags.FLAGS.__flags)

    if not os.path.exists(FLAGS.checkpoint_dir):
        os.makedirs(FLAGS.checkpoint_dir)
    if not os.path.exists(FLAGS.sample_dir):
        os.makedirs(FLAGS.sample_dir)
    if not os.path.exists(FLAGS.gen_dir):
        os.makedirs(FLAGS.gen_dir)

    with tf.Session() as sess:
        if FLAGS.dataset == 'MidiNet_vg':
            model = MidiNet(sess, batch_size=FLAGS.batch_size, output_w=FLAGS.output_w, output_h=FLAGS.output_h, c_dim=FLAGS.c_dim,
                    dataset_name=FLAGS.dataset, is_crop=FLAGS.is_crop, checkpoint_dir=FLAGS.checkpoint_dir, sample_dir=FLAGS.sample_dir, 
                            gen_dir=FLAGS.gen_dir)
        
        if FLAGS.is_train:
            model.train(FLAGS)
        else:
            model.load(FLAGS.checkpoint_dir)

        

if __name__ == '__main__':
    tf.app.run()

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


{'batch_size': <absl.flags._flag.Flag object at 0x000002D7CD119550>,
 'beta1': <absl.flags._flag.Flag object at 0x000002D7C753C6A0>,
 'c_dim': <absl.flags._flag.Flag object at 0x000002D7CD119668>,
 'checkpoint_dir': <absl.flags._flag.Flag object at 0x000002D7CD119710>,
 'dataset': <absl.flags._flag.Flag object at 0x000002D7CD119860>,
 'epoch': <absl.flags._flag.Flag object at 0x000002D7C753C668>,
 'gen_dir': <absl.flags._flag.Flag object at 0x000002D7CD119A58>,
 'generation_test': <absl.flags._flag.BooleanFlag object at 0x000002D7CD119A90>,
 'h': <tensorflow.python.platform.app._HelpFlag object at 0x000002D7CD119B70>,
 'help': <tensorflow.python.platform.app._HelpFlag object at 0x000002D7CD119B70>,
 'helpfull': <tensorflow.python.platform.app._HelpfullFlag object at 0x000002D7CD119C18>,
 'helpshort': <tensorflow.python.platform.app._HelpshortFlag object at 0x000002D7CD119C88>,
 'is_crop': <absl.flags._flag.BooleanFlag object at 0x000002D7CD1199E8>,
 'is_train': <absl.flags._flag.Boolea

Epoch: [ 2] [   7/  40] time: 11.1063, d_loss: 0.88233387, g_loss: 15.85787201
Epoch: [ 2] [   8/  40] time: 11.1951, d_loss: 0.84929264, g_loss: 16.30151558
Epoch: [ 2] [   9/  40] time: 11.2829, d_loss: 0.87850249, g_loss: 12.27997684
Epoch: [ 2] [  10/  40] time: 11.3706, d_loss: 0.89652514, g_loss: 14.62971115
Epoch: [ 2] [  11/  40] time: 11.4574, d_loss: 0.88147342, g_loss: 14.31614590
Epoch: [ 2] [  12/  40] time: 11.5452, d_loss: 0.89359391, g_loss: 13.03873634
Epoch: [ 2] [  13/  40] time: 11.6339, d_loss: 0.92454100, g_loss: 14.23942089
Epoch: [ 2] [  14/  40] time: 11.7217, d_loss: 0.94127238, g_loss: 12.61321068
Epoch: [ 2] [  15/  40] time: 11.8095, d_loss: 0.96250516, g_loss: 15.02544212
Epoch: [ 2] [  16/  40] time: 11.8972, d_loss: 0.92553622, g_loss: 13.52936459
Epoch: [ 2] [  17/  40] time: 11.9860, d_loss: 0.88475698, g_loss: 12.44156647
Epoch: [ 2] [  18/  40] time: 12.0757, d_loss: 0.87479913, g_loss: 13.44261074
Epoch: [ 2] [  19/  40] time: 12.1645, d_loss: 0.888

Epoch: [ 4] [  31/  40] time: 20.3735, d_loss: 0.67068875, g_loss: 9.20493126
Epoch: [ 4] [  32/  40] time: 20.4623, d_loss: 0.67200786, g_loss: 9.19070244
Epoch: [ 4] [  33/  40] time: 20.5600, d_loss: 0.70089865, g_loss: 8.43228626
Epoch: [ 4] [  34/  40] time: 20.6508, d_loss: 0.68103850, g_loss: 9.67306614
Epoch: [ 4] [  35/  40] time: 20.7366, d_loss: 0.68228543, g_loss: 8.46322536
Epoch: [ 4] [  36/  40] time: 20.8273, d_loss: 0.63204104, g_loss: 9.34232998
Epoch: [ 4] [  37/  40] time: 20.9221, d_loss: 0.65861773, g_loss: 8.82865143
Epoch: [ 4] [  38/  40] time: 21.0118, d_loss: 0.65376914, g_loss: 9.07763767
Epoch: [ 4] [  39/  40] time: 21.1036, d_loss: 0.65561128, g_loss: 8.37088299
Epoch: [ 4] time: 21.1036, d_loss: 0.01639028
Epoch: [ 5] [   0/  40] time: 21.1923, d_loss: 0.68361807, g_loss: 7.89057684
[Sample] d_loss: 0.67439741, g_loss: 8.02258778
Epoch: [ 5] [   1/  40] time: 21.3166, d_loss: 0.63416904, g_loss: 8.90304852
Epoch: [ 5] [   2/  40] time: 21.4074, d_loss: 0

Epoch: [ 7] [  14/  40] time: 29.6633, d_loss: 0.47365254, g_loss: 8.01934433
Epoch: [ 7] [  15/  40] time: 29.7520, d_loss: 0.47924694, g_loss: 9.63057995
Epoch: [ 7] [  16/  40] time: 29.8428, d_loss: 0.48146504, g_loss: 8.08001614
Epoch: [ 7] [  17/  40] time: 29.9316, d_loss: 0.48501045, g_loss: 7.02933598
Epoch: [ 7] [  18/  40] time: 30.0203, d_loss: 0.48417678, g_loss: 8.53053188
Epoch: [ 7] [  19/  40] time: 30.1091, d_loss: 0.50309390, g_loss: 8.56275845
Epoch: [ 7] [  20/  40] time: 30.1978, d_loss: 0.49762088, g_loss: 8.10423851
[Sample] d_loss: 0.51249427, g_loss: 7.76082897
Epoch: [ 7] [  21/  40] time: 30.3135, d_loss: 0.49236852, g_loss: 7.64793015
Epoch: [ 7] [  22/  40] time: 30.4053, d_loss: 0.46853036, g_loss: 7.62878180
Epoch: [ 7] [  23/  40] time: 30.4980, d_loss: 0.47301757, g_loss: 7.39971256
Epoch: [ 7] [  24/  40] time: 30.5988, d_loss: 0.48359039, g_loss: 10.26872158
Epoch: [ 7] [  25/  40] time: 30.6935, d_loss: 0.49559429, g_loss: 7.70686483
Epoch: [ 7] [  

...

Epoch: [43] [  27/  40] time: 161.3722, d_loss: 0.37059176, g_loss: 7.35585642
Epoch: [43] [  28/  40] time: 161.4619, d_loss: 0.36773267, g_loss: 8.45773506
Epoch: [43] [  29/  40] time: 161.5527, d_loss: 0.37003219, g_loss: 8.24553776
Epoch: [43] [  30/  40] time: 161.6474, d_loss: 0.36511451, g_loss: 8.06044292
Epoch: [43] [  31/  40] time: 161.7392, d_loss: 0.37498841, g_loss: 7.96249723
Epoch: [43] [  32/  40] time: 161.8269, d_loss: 0.37946856, g_loss: 7.45746756
Epoch: [43] [  33/  40] time: 161.9128, d_loss: 0.37555549, g_loss: 7.26046705
Epoch: [43] [  34/  40] time: 162.0015, d_loss: 0.37145102, g_loss: 7.46906567
Epoch: [43] [  35/  40] time: 162.0892, d_loss: 0.36651325, g_loss: 8.29128933
Epoch: [43] [  36/  40] time: 162.1770, d_loss: 0.37255311, g_loss: 8.42477226
Epoch: [43] [  37/  40] time: 162.2668, d_loss: 0.37117416, g_loss: 7.94238186
Epoch: [43] [  38/  40] time: 162.3565, d_loss: 0.38110632, g_loss: 7.43449640
Epoch: [43] [  39/  40] time: 162.4473, d_loss: 0.36

Epoch: [46] [  10/  40] time: 170.8139, d_loss: 0.37713218, g_loss: 8.10643387
Epoch: [46] [  11/  40] time: 170.9047, d_loss: 0.37688732, g_loss: 8.19415569
Epoch: [46] [  12/  40] time: 170.9924, d_loss: 0.36376435, g_loss: 7.70076656
Epoch: [46] [  13/  40] time: 171.0941, d_loss: 0.37094378, g_loss: 7.78146315
Epoch: [46] [  14/  40] time: 171.1839, d_loss: 0.36743429, g_loss: 7.92080259
Epoch: [46] [  15/  40] time: 171.2787, d_loss: 0.37352031, g_loss: 8.11615372
Epoch: [46] [  16/  40] time: 171.3704, d_loss: 0.36428672, g_loss: 8.06662846
Epoch: [46] [  17/  40] time: 171.4592, d_loss: 0.36865640, g_loss: 7.93067741
Epoch: [46] [  18/  40] time: 171.5459, d_loss: 0.36937118, g_loss: 8.02062798
Epoch: [46] [  19/  40] time: 171.6337, d_loss: 0.37132847, g_loss: 7.96905756
Epoch: [46] [  20/  40] time: 171.7354, d_loss: 0.36170566, g_loss: 8.47583199
Epoch: [46] [  21/  40] time: 171.8431, d_loss: 0.37021971, g_loss: 7.69024324
Epoch: [46] [  22/  40] time: 171.9528, d_loss: 0.35

Epoch: [48] [  32/  40] time: 180.4483, d_loss: 0.36688951, g_loss: 8.02458954
Epoch: [48] [  33/  40] time: 180.5410, d_loss: 0.36874446, g_loss: 7.73783922
Epoch: [48] [  34/  40] time: 180.6308, d_loss: 0.36800349, g_loss: 8.31874371
Epoch: [48] [  35/  40] time: 180.7175, d_loss: 0.37199384, g_loss: 8.44397545
Epoch: [48] [  36/  40] time: 180.8063, d_loss: 0.36384648, g_loss: 8.61163902
Epoch: [48] [  37/  40] time: 180.9030, d_loss: 0.36709547, g_loss: 8.23145103
Epoch: [48] [  38/  40] time: 180.9988, d_loss: 0.35946670, g_loss: 8.35153008
Epoch: [48] [  39/  40] time: 181.0957, d_loss: 0.36877614, g_loss: 7.89210367
Epoch: [48] time: 181.0957, d_loss: 0.00921940
Epoch: [49] [   0/  40] time: 181.1841, d_loss: 0.37208572, g_loss: 7.71782970
Epoch: [49] [   1/  40] time: 181.2718, d_loss: 0.36947221, g_loss: 8.30824471
Epoch: [49] [   2/  40] time: 181.3592, d_loss: 0.36999124, g_loss: 8.57036304
Epoch: [49] [   3/  40] time: 181.4490, d_loss: 0.37023973, g_loss: 8.18211269
Epoch

SystemExit: 

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2918: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
!tensorboard --logdir /logs

In [ ]:
# !pip install music21

In [2]:
folder_png ='C:/Users/Gustavo/Documents/GitHub/GAN_project_PUCP/Midinet_VG/Model_type1/samples'
import os
import numpy as np
import matplotlib.pyplot as plt
import music21

In [3]:
def generate_midis(png_dir):
    for f in os.listdir(png_dir):    
        image_plt = plt.imread(os.path.join(png_dir,f))
        image_arr = np.array(image_plt)
        r_song = []
        for i in range(0,640,128):
            print(i)
            r_measure = []
            for j in range(0,16):
                measure = image_arr[j][i:i+128]
                #print(len(measure))
                r_measure.append([0 if x[0]<0.65 else x[0] for x in measure])
            r_song.append(r_measure)
        new_test_stream = music21.stream.Stream()
        sup = r_song
        new_track = music21.stream.Part()
        for j in range(0,len(sup)):
            measure = sup[j]
            for k in range(0,len(measure)):
                if sum(measure[k]) == 0 :
                    new_track.append(music21.note.Rest(type='16th'))
                else:
                    note_value = np.argmax(measure[k])
                    new_track.append(music21.note.Note(note_value, type='16th'))
        new_test_stream.append(new_track)
        new_test_stream.write('midi',f.split('.png')[0]+'.mid')
        new_test_stream.show('midi') 

In [4]:
generate_midis(folder_png)

0
128
256
384
512


0
128
256
384
512


0
128
256
384
512


0
128
256
384
512


0
128
256
384
512


...

0
128
256
384
512


0
128
256
384
512


0
128
256
384
512


0
128
256
384
512


0
128
256
384
512
